<a href="https://colab.research.google.com/github/BrendaGilisho/ml-predict/blob/main/Model_Development_and_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')
import re
def load_doc(filename):
# open the file as read only
    file = open(filename, 'r' )
# read all text
    text = file.read()
# close the file
    file.close()
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def clean_doc(doc):
# split into tokens by white space
    tokens = doc.split()
# prepare regex for char filtering
    re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
# remove punctuation from each word
    tokens = [re_punc.sub( '' , w) for w in tokens]
# remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
    stop_words = set(stopwords.words( 'english' ))
    tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [4]:
filename = '/content/drive/My Drive/ML/twitter/test.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

['years', 'since', 'diagnosed', 'Today', 'taking', 'moment', 'reflect', 'far', 'come', 'need', 'planning', 'spend', 'little', 'time', 'possible', 'Awake', 'need', 'sleep', 'brain', 'RT', 'bears', 'make', 'perfect', 'gifts', 'great', 'beginners', 'Get', 'stitching', 'Sew', 'sale', 'hard', 'say', 'whether', 'packing', 'lists', 'making', 'life', 'easier', 'reinforcing', 'much', 'still', 'needs', 'Making', 'packing', 'lists', 'new', 'At', 'point', 'keeping', 'stuff', 'nostalgic', 'reasons', 'cross', 'line', 'plain', 'old', 'Currently', 'packing', 'think', 'closet', 'bothered', 'take', 'away', 'way', 'RT', 'ITV', 'releases', 'promo', 'video', 'final', 'series', 'Downton', 'Abbey', 'much', 'way', 'much', 'never', 'want', 'put', 'one', 'together', 'Moving', 'stuff', 'still', 'loads', 'Back', 'moving', 'peaceful', 'FFS', 'come', 'much', 'way', 'Massive', 'purge', 'Hideous', 'traffic', 'Must', 'remember', 'pay', 'closer', 'attention', 'traffic', 'Packing', 'Feels', 'good', 'In', 'looking', 'int

In [5]:
import string
import re
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
# load doc
    doc = load_doc(filename)
# clean doc
    tokens = clean_doc(doc)
# update counts
    vocab.update(tokens)
# load all docs in a directory
def process_docs(directory, vocab):
# walk through all files in the folder
    for filename in listdir(directory):
# skip any reviews in the test set
        if filename.startswith( 'tweet' ):
            continue
# create the full path of the file to open
        path = directory + '/' + filename
# add doc to vocab
        add_doc_to_vocab(path, vocab)

In [6]:
# define vocab
vocab = Counter()
# add all docs to vocab
process_docs( '/content/drive/My Drive/ML/twitter' , vocab)
process_docs( '/content/drive/My Drive/ML/twitter' , vocab)
# print the size of the vocab
print(len(vocab))

2259


In [7]:
# # print the top words in the vocab
print(vocab.most_common(50))
# # keep tokens with a min occurrence
min_occurane = 5
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))
def save_list(lines, filename):
    data = ' \n ' .join(lines)
# open file
    file = open(filename, 'w' )
# write text
    file.write(data)
# close file
    file.close()

# # save tokens to a vocabulary file
save_list(tokens, 'vocab.txt' )

[('The', 110), ('What', 90), ('RT', 88), ('How', 84), ('like', 70), ('know', 70), ('love', 56), ('Do', 52), ('You', 48), ('get', 46), ('people', 46), ('Have', 46), ('one', 40), ('good', 40), ('never', 36), ('much', 32), ('think', 32), ('If', 32), ('would', 32), ('make', 30), ('want', 28), ('right', 28), ('feel', 28), ('work', 28), ('someone', 28), ('still', 26), ('new', 26), ('Why', 26), ('go', 26), ('man', 26), ('need', 24), ('ever', 24), ('Is', 24), ('Mark', 24), ('life', 22), ('take', 22), ('many', 22), ('day', 22), ('always', 22), ('Your', 22), ('thing', 22), ('miss', 22), ('When', 22), ('time', 20), ('hard', 20), ('way', 20), ('makes', 20), ('We', 20), ('two', 20), ('It', 20)]
309


In [8]:
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())


In [9]:
# turn a doc into clean tokens
def clean_doc(doc, vocab):
# split into tokens by white space
    tokens = doc.split()
# prepare regex for char filtering
    re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
# remove punctuation from each word
    tokens = [re_punc.sub( '' , w) for w in tokens]
# filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' ' .join(tokens)
    return tokens

In [10]:
# load all docs in a directory
def process_docs(directory, vocab, is_train):
    documents = list()
# walk through all files in the folder
    for filename in listdir(directory):
# skip any reviews in the test set
        if is_train and filename.startswith( 'tweet' ):
            continue
        if not is_train and not filename.startswith( 'tweet' ):
            continue
# create the full path of the file to open
        path = directory + '/' + filename
# load the doc
        doc = load_doc(path)
# clean doc
        tokens = clean_doc(doc, vocab)
# add to list
        documents.append(tokens)
    return documents


In [11]:
import string
import re
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
def load_clean_dataset(vocab, is_train):
    neg = process_docs( '/content/drive/My Drive/ML/twitter' , vocab, is_train)
    pos = process_docs( '/content/drive/My Drive/ML/twitter' , vocab, is_train)
    docs = neg + pos
    labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
    return docs, labels

In [12]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [13]:
def encode_docs(tokenizer, max_length, docs):
# integer encode
    encoded = tokenizer.texts_to_sequences(docs)
# pad sequences
    padded = pad_sequences(encoded, maxlen=max_length, padding= 'post' )
    return padded

In [14]:
# define the model
def define_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_length))
    model.add(Conv1D(filters=32, kernel_size=8, activation= 'relu' ))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation= 'relu'))
    model.add(Dense(1, activation= 'sigmoid' ))
# compile network
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
# summarize defined model
    model.summary()
    plot_model(model, to_file= 'model.png' , show_shapes=True)
    return model

In [15]:
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())
# load training data
train_docs, ytrain = load_clean_dataset(vocab, True)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# # define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
# print( ' Vocabulary size: %d ' % vocab_size)
# # calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print( ' Maximum length: %d ' % max_length)
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
# define model
model = define_model(vocab_size, max_length)

 Maximum length: 2056 
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2056, 100)         30000     
                                                                 
 conv1d (Conv1D)             (None, 2049, 32)          25632     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1024, 32)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 50)                1638450   
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                 

In [16]:
model.fit(Xtrain, ytrain, epochs=10, verbose=2)


Epoch 1/10
1/1 - 1s - loss: 0.6936 - accuracy: 0.5000 - 1s/epoch - 1s/step
Epoch 2/10
1/1 - 0s - loss: 1.1801 - accuracy: 0.5000 - 46ms/epoch - 46ms/step
Epoch 3/10
1/1 - 0s - loss: 0.7314 - accuracy: 0.5000 - 45ms/epoch - 45ms/step
Epoch 4/10
1/1 - 0s - loss: 0.6931 - accuracy: 0.5000 - 44ms/epoch - 44ms/step
Epoch 5/10
1/1 - 0s - loss: 0.6931 - accuracy: 0.5000 - 44ms/epoch - 44ms/step
Epoch 6/10
1/1 - 0s - loss: 0.6931 - accuracy: 0.5000 - 44ms/epoch - 44ms/step
Epoch 7/10
1/1 - 0s - loss: 0.6932 - accuracy: 0.5000 - 43ms/epoch - 43ms/step
Epoch 8/10
1/1 - 0s - loss: 0.6931 - accuracy: 0.5000 - 44ms/epoch - 44ms/step
Epoch 9/10
1/1 - 0s - loss: 0.6931 - accuracy: 0.5000 - 46ms/epoch - 46ms/step
Epoch 10/10
1/1 - 0s - loss: 0.6931 - accuracy: 0.5000 - 44ms/epoch - 44ms/step


In [17]:
model.save( 'model.h5' )

In [18]:
# load all reviews
from keras.models import load_model

test_docs, ytest = load_clean_dataset(vocab, False)
# create the tokenizer
# encode data
Xtest = encode_docs(tokenizer, max_length, test_docs)
model = load_model( 'model.h5' )
# _, acc = model.evaluate(Xtrain, ytrain, verbose=0)
# print( ' Train Accuracy: %f ' % (acc*100))
_, acc = model.evaluate(Xtest, ytest, verbose=0)
print( ' Test Accuracy: %f ' % (acc*100))


 Test Accuracy: 50.000000 


In [19]:
import string
import re
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import load_model
# classify a review as negative or positive
def predict_sentiment(review, vocab, tokenizer, max_length, model):
# clean review
    line = clean_doc(review, vocab)
# encode and pad review
    padded = encode_docs(tokenizer, max_length, [line])
# predict sentiment
    yhat = model.predict(padded, verbose=0)
# retrieve predicted percentage and label
    percent_pos = yhat[0,0]
    if round(percent_pos) == 0:
        return (1-percent_pos), 'Depression'
    return percent_pos, 'No Depression'

In [20]:
text = ' Feeling a strange mix of excited and sad. Such a big change looming '
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print( ' Review: [%s]\nSentiment: %s (%.3f%%) ' % (text, sentiment, percent*100))
# test no depression
text = ' why is this their go to song agsdhsjjs i mean SAME but '
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print( ' Review: [%s]\nSentiment: %s (%.3f%%) ' % (text, sentiment, percent*100))


 Review: [ Feeling a strange mix of excited and sad. Such a big change looming ]
Sentiment: No Depression (50.138%) 
 Review: [ why is this their go to song agsdhsjjs i mean SAME but ]
Sentiment: No Depression (50.150%) 
